## DOWNLOADS

In [ ]:
!pip install elasticsearch
!pip install spacy tqdm
!python -m spacy download en_core_web_sm
!pip install pandas textblob vaderSentiment spacy
!python -m textblob.download_corpora

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 2.1 MB/s eta 0:00:00


## LIBRARY IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

import itertools
import re
import string
import pickle
import os
import json
import spacy
import inflect

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.model_selection import KFold
from matplotlib import style

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.corpus import stopwords, wordnet
from wordcloud import WordCloud
from copy import deepcopy
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from time import time

from IPython.display import (
    Markdown as md,
    Latex,
    HTML,
)
from tqdm import tqdm
from tqdm.auto import tqdm

from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# set plot style
sns.set()
%matplotlib inline

from google.colab import drive

drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Mounted at /content/drive


## LOAD DATASETS

In [ ]:
# Specifying the directory where I want to save the files
save_directory = "/content/drive/MyDrive/nlp data/Data/"

# Ensuring the directory exists
os.makedirs(save_directory, exist_ok=True)

In [ ]:
player_detail = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of player_detail.csv')
transfer_history = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of tm_player_transfer_history.csv')
player_market_value_development = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of tm_player_market_value_development.csv')

In [ ]:
# Replace 'a998-102-88-36-153.ngrok-free.app' with my actual ngrok URL
ngrok_url = "https://c1cd-196-43-215-222.ngrok-free.app"
username = "elastic"
password = "changeme"  # Replace with the actual password

# Connect to the Elasticsearch instance
# with authentication and SSL verification
es = Elasticsearch(
    ngrok_url,
    basic_auth=(username, password),
    verify_certs=True  # Enable SSL certificate verification
)

# Verify the connection
try:
    info = es.info().body
    print("Connected to Elasticsearch!")
    print(json.dumps(info, indent=4))
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")

Connected to Elasticsearch!
{
    "name": "52b00a9c2bdf",
    "cluster_name": "docker-cluster",
    "cluster_uuid": "MaiorFAUSpuMsi-zUpRjxw",
    "version": {
        "number": "8.6.2",
        "build_flavor": "default",
        "build_type": "docker",
        "build_hash": "2d58d0f136141f03239816a4e360a8d17b6d8f29",
        "build_date": "2023-02-13T09:35:20.314882762Z",
        "build_snapshot": false,
        "lucene_version": "9.4.2",
        "minimum_wire_compatibility_version": "7.17.0",
        "minimum_index_compatibility_version": "7.0.0"
    },
    "tagline": "You Know, for Search"
}


In [ ]:
# Function to sanitize filenames by removing invalid characters
def sanitize_filename(filename):
    # Replace invalid characters with underscores
    return re.sub(r'[\\/*?:"<>|]', "_", filename)

# Function to download all documents from a specific index and save them to a single file
def download_all_documents_to_single_file(index_name, save_dir):
    try:
        # Using the scan helper to iterate through all documents in the index
        documents = scan(es, index=index_name, query={"query": {"match_all": {}}})

        all_docs = []

        for doc in documents:
            doc_source = doc['_source']
            all_docs.append(doc_source)

        # Sanitizing the index name to create a valid filename
        sanitized_index_name = sanitize_filename(index_name)
        filename = f'{sanitized_index_name}_all_docs.json'
        file_path = os.path.join(save_dir, filename)

        # Save all documents to a single file
        with open(file_path, 'w') as f:
            json.dump(all_docs, f, indent=4)

        print(f"All documents from index '{index_name}' have been downloaded and saved to {file_path}.")
    except Exception as e:
        print(f"Error downloading documents from index '{index_name}': {e}")

In [ ]:
# Download all documents from the "news_feed" index and save to a single file
download_all_documents_to_single_file("news_feed", save_directory)

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/elastic_transport/_transport.py", line 342, in perform_request
    resp = node.perform_request(
  File "/usr/local/lib/python3.10/dist-packages/elastic_transport/_node/_http_urllib3.py", line 202, in perform_request
    raise err from None
elastic_transport.ConnectionError: Connection error caused by: ProtocolError(('Connection broken: IncompleteRead(5341038 bytes read, 2739498 more expected)', IncompleteRead(5341038 bytes read, 2739498 more expected)))


Error downloading documents from index 'news_feed': AuthorizationException(403, 'This ngrok account has reached its network bandwidth limit for the month. Please log into https://dashboard.ngrok.com to view your options.\r\n\r\nERR_NGROK_725\r\n')


In [ ]:
# Download all documents from the "miao_feed" index and save to a single file
download_all_documents_to_single_file("miao_feed", save_directory)

NameError: name 'download_all_documents_to_single_file' is not defined

In [ ]:
# Replace 'a998-102-88-36-153.ngrok-free.app' with the actual ngrok URL
ngrok_url = "https://c1cd-196-43-215-222.ngrok-free.app"
username = "elastic"
password = "changeme"  # Replace with the actual password

# Connect to the Elasticsearch instance
# with authentication and SSL verification
es = Elasticsearch(
    ngrok_url,
    basic_auth=(username, password),
    verify_certs=True  # Enable SSL certificate verification
)

# Verify the connection
try:
    info = es.info().body
    print("Connected to Elasticsearch!")
    print(json.dumps(info, indent=4))
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")

# Perform a sample query
try:
    news_feed_response = es.search(
        index="news_feed",
        query={"match_all": {}}).body  # Access the body attribute
    print("Search query response:")
    print(json.dumps(
        news_feed_response,
        indent=4))  # Pretty print the search query response

    # Save the JSON response to a file
    with open('news_feed_response.json', 'w') as f:
        json.dump(news_feed_response, f, indent=4)
    print("Response saved to response.json")
except Exception as e:
    print(f"Error performing search query: {e}")

Connected to Elasticsearch!
{
    "name": "52b00a9c2bdf",
    "cluster_name": "docker-cluster",
    "cluster_uuid": "MaiorFAUSpuMsi-zUpRjxw",
    "version": {
        "number": "8.6.2",
        "build_flavor": "default",
        "build_type": "docker",
        "build_hash": "2d58d0f136141f03239816a4e360a8d17b6d8f29",
        "build_date": "2023-02-13T09:35:20.314882762Z",
        "build_snapshot": false,
        "lucene_version": "9.4.2",
        "minimum_wire_compatibility_version": "7.17.0",
        "minimum_index_compatibility_version": "7.0.0"
    },
    "tagline": "You Know, for Search"
}
Search query response:
{
    "took": 15,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 10000,
            "relation": "gte"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "news_feed",
                "_id": "

In [ ]:
# Replace 'a998-102-88-36-153.ngrok-free.app' with the actual ngrok URL
ngrok_url = "https://ab80-102-89-45-153.ngrok-free.app"
username = "elastic"
password = "changeme"  # Replace with the actual password

# Connect to the Elasticsearch instance
# with authentication and SSL verification
es = Elasticsearch(
    ngrok_url,
    basic_auth=(username, password),
    verify_certs=True  # Enable SSL certificate verification
)

# Verify the connection
try:
    info = es.info().body
    print("Connected to Elasticsearch!")
    print(json.dumps(info, indent=4))
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")

# Perform a sample query
try:
    miao_feed_response = es.search(
        index="miao_feed",
        query={"match_all": {}}).body  # Access the body attribute
    print("Search query response:")
    print(json.dumps(
        miao_feed_response,
        indent=4))  # Pretty print the search query response

    # Save the JSON response to a file
    with open('miao_feed_response.json', 'w') as f:
        json.dump(miao_feed_response, f, indent=4)
    print("Response saved to response.json")
except Exception as e:
    print(f"Error performing search query: {e}")


Connected to Elasticsearch!
{
    "name": "52b00a9c2bdf",
    "cluster_name": "docker-cluster",
    "cluster_uuid": "MaiorFAUSpuMsi-zUpRjxw",
    "version": {
        "number": "8.6.2",
        "build_flavor": "default",
        "build_type": "docker",
        "build_hash": "2d58d0f136141f03239816a4e360a8d17b6d8f29",
        "build_date": "2023-02-13T09:35:20.314882762Z",
        "build_snapshot": false,
        "lucene_version": "9.4.2",
        "minimum_wire_compatibility_version": "7.17.0",
        "minimum_index_compatibility_version": "7.0.0"
    },
    "tagline": "You Know, for Search"
}
Search query response:
{
    "took": 62,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 10000,
            "relation": "gte"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "miao_feed",
                "_id": "

### CHECK DATA

In [ ]:
player_detail.head()

,Name,player_id,DOB,Age,Foot,Height (m),Nationality,Citizenship,Position,Other positions,Team,Joined,Contract expires
0,Thibaut Courtois,108390,1992-05-11,31.0,left,2.00,Belgium,['Belgium'],Goalkeeper,NaN,Real Madrid,2018-08-09,2026-06-30
1,Andriy Lunin,404839,1999-02-11,25.0,right,1.91,Ukraine,['Ukraine'],Goalkeeper,NaN,Real Madrid,2018-07-01,2025-06-30
2,Kepa Arrizabalaga,192279,1994-10-03,29.0,right,1.88,Spain,['Spain'],Goalkeeper,NaN,Real Madrid,2023-08-14,2024-06-30
3,Éder Militão,401530,1998-01-18,26.0,right,1.86,Brazil,"['Brazil', 'Spain']",Centre-Back,Right-Back,Real Madrid,2019-07-01,2028-06-30
4,David Alaba,59016,1992-06-24,31.0,left,1.80,Austria,['Austria'],Centre-Back,Left-Back,Real Madrid,2021-07-01,2026-06-30


In [ ]:
transfer_history.head()

,from_clubName,from_latitude,from_longitude,to_clubName,to_latitude,to_longitude,url,futureTransfer,date,dateUnformatted,upcoming,season,marketValue,fee,player_id,Total transfer fees,TM Market Value,Transfer Type,Transfer Fee
0,Chelsea,51.477520,-0.201590,Real Madrid,40.452357,-3.686789,/thibaut-courtois/transfers/spieler/108390,0.0,2018-08-09,2018-08-09,False,18/19,€65.00m,€35.00m,108390,45150000,65000000.0,Transfer,35000000.0
1,Atlético Madrid,40.401730,-3.720635,Chelsea,51.477520,-0.201590,/thibaut-courtois/transfers/spieler/108390,0.0,2014-06-30,2014-06-30,False,13/14,€25.00m,End of loan,108390,45150000,25000000.0,End of Loan,0.0
2,Chelsea,51.477520,-0.201590,Atlético Madrid,40.401730,-3.720635,/thibaut-courtois/transfers/spieler/108390,0.0,2011-07-27,2011-07-27,False,11/12,€4.00m,"Loan fee:<br /><i class=""normaler-text"">€1.20m...",108390,45150000,4000000.0,Loan,1200000.0
3,KRC Genk,51.005026,5.533332,Chelsea,51.477520,-0.201590,/thibaut-courtois/transfers/spieler/108390,0.0,2011-07-26,2011-07-26,False,11/12,€4.00m,€8.95m,108390,45150000,4000000.0,Transfer,8950000.0
4,KRC Genk U19,51.005026,5.533332,KRC Genk,51.005026,5.533332,/thibaut-courtois/transfers/spieler/108390,0.0,2009-07-01,2009-07-01,False,09/10,€50k,-,108390,45150000,50000.0,Transfer,0.0


In [ ]:
player_market_value_development.head()

,mw,datum_mw,verein,age,player_id,Highest market value,highest_date,last_change
0,50000.0,2009-05-27,KRC Genk U19,17,108390,75000000.0,2020-10-08,2024-03-21
1,150000.0,2010-07-28,KRC Genk,18,108390,75000000.0,2020-10-08,2024-03-21
2,1000000.0,2010-10-06,KRC Genk,18,108390,75000000.0,2020-10-08,2024-03-21
3,2000000.0,2010-12-28,KRC Genk,18,108390,75000000.0,2020-10-08,2024-03-21
4,3000000.0,2011-05-24,KRC Genk,19,108390,75000000.0,2020-10-08,2024-03-21


## DATA EXTRACTION FROM DATABASE

In [ ]:
from elasticsearch import Elasticsearch
import json
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Replace with your actual ngrok URL and Elasticsearch credentials
ngrok_url = "https://ab80-102-89-45-153.ngrok-free.app"
username = "elastic"
password = "changeme"

# Connect to Elasticsearch with authentication and SSL verification, and set timeout
es = Elasticsearch(
    ngrok_url,
    basic_auth=(username, password),
    verify_certs=True,
    timeout=60,  # Increase timeout to 60 seconds
    max_retries=10,  # Set maximum retries
    retry_on_timeout=True
)

# Verify the connection
try:
    info = es.info().body
    logger.info("Connected to Elasticsearch!")
    logger.info(json.dumps(info, indent=4))  # Print Elasticsearch info
except Exception as e:
    logger.error(f"Error connecting to Elasticsearch: {e}")

# Function to retrieve all documents from the news_feed index
def retrieve_all_documents():
    try:
        # Use Elasticsearch scroll API to retrieve all documents
        response = es.search(index="news_feed", scroll="2m", body={"query": {"match_all": {}}, "size": 100})
        scroll_id = response['_scroll_id']
        all_documents = response['hits']['hits']
        logger.info(f"Initial batch size: {len(all_documents)} documents")

        while True:
            logger.info(f"Fetching next batch of documents with scroll_id: {scroll_id}")
            response = es.scroll(scroll_id=scroll_id, scroll='2m')
            hits = response['hits']['hits']
            logger.info(f"Batch size: {len(hits)} documents")

            if not hits:
                logger.info("No more documents to retrieve.")
                break

            all_documents.extend(hits)
            logger.info(f"Total documents retrieved so far: {len(all_documents)}")

        return all_documents

    except Exception as e:
        logger.error(f"Error retrieving documents: {e}")
        return None

# Retrieve all documents from news_feed index
try:
    all_documents = retrieve_all_documents()

    if all_documents is not None:
        # Print total number of documents retrieved
        logger.info(f"Total documents retrieved: {len(all_documents)}")

        # Example: Print the first document
        if len(all_documents) > 0:
            first_document = all_documents[0]
            logger.info("First document:")
            logger.info(json.dumps(first_document, indent=4))

        # Example: Access specific fields in documents
        for document in all_documents:
            title = document["_source"].get("title", "No title")
            url = document["_source"].get("url", "No URL")
            logger.info(f"Title: {title}, URL: {url}")

        # Save the JSON response to a file
        with open('news_feed_documents.json', 'w') as f:
            json.dump(all_documents, f, indent=4)
        logger.info("Documents saved to news_feed_documents.json")

    else:
        logger.warning("No documents retrieved.")

except Exception as e:
    logger.error(f"Error processing documents: {e}")

<ipython-input-9-9d90b06d8eb7>:16: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(
ERROR:root:Error retrieving documents: AuthorizationException(403, 'This ngrok account has reached its network bandwidth limit for the month. Please log into https://dashboard.ngrok.com to view your options.\r\n\r\nERR_NGROK_725\r\n')


## DATA CLEANING

### NEWS DATA

In [ ]:
player_detail = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of player_detail.csv')
transfer_history = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of tm_player_transfer_history.csv')
player_market_value_development = pd.read_csv('/content/drive/MyDrive/nlp data/Copy of tm_player_market_value_development.csv')

miao_feed_all_docs = pd.read_csv('/content/drive/MyDrive/nlp data/Data/miao_feed_all_docs.csv')
news_feed_all_docs = pd.read_csv('/content/drive/MyDrive/nlp data/Data/news_feed_all_docs.csv')

In [ ]:
miao_feed_all_docs.head(2)

In [ ]:
# Drop multiple columns
miao_feed_all_docs_dropped = miao_feed_all_docs.drop(columns=[
    'mini_summary_cn', 'summary_cn', 'title_cn',
    'file_name', 'chunks', 'translations', 'video_generated', 'mini_summary', 'title_image', 'summary'])

In [ ]:
# Drop multiple columns
news_feed_all_docs_dropped = news_feed_all_docs.drop(
    columns=['video_generated', 'file_name'])

In [ ]:
news_feed_all_docs_dropped.isna().sum()

In [ ]:
def clean_content(text):
    # Remove HTML tags
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    # Remove "[image]" from the text
    text = re.sub(r'\[image\]', '', text)

    # Remove "(Photo: ...)" where "photo:" is case insensitive
    text = re.sub(r'\(photo:\s*[^)]*\)', '', text, flags=re.IGNORECASE)

    # Remove URLs
    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(url_pattern, '', text)

    # Remove excessive values in between text (adjust as per your specific case)
    text = re.sub(r'\b(?:excessive_value1|excessive_value2)\b', '', text, flags=re.IGNORECASE)

    clean_text = re.sub(r'\s+', ' ', text).strip()

    return clean_text

In [ ]:
miao_feed_all_docs_dropped['content'] = miao_feed_all_docs_dropped['content'].apply(clean_content)
news_feed_all_docs_dropped['content'] = news_feed_all_docs_dropped['content'].apply(clean_content)

In [ ]:
# miao_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/nlp data/Data/miao_feed_all_docs_dropped.csv', index=False)
# news_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/nlp data/Data/news_feed_all_docs_dropped.csv', index=False)

### PLAYER DATA

In [ ]:
# Cleaning transfer_history
# transfer_history['from_latitude'].fillna(0, inplace=True)
# transfer_history['from_longitude'].fillna(0, inplace=True)
# transfer_history['to_latitude'].fillna(0, inplace=True)
# transfer_history['to_longitude'].fillna(0, inplace=True)
transfer_history['date'].fillna('1900-01-01', inplace=True)
transfer_history['season'].fillna('Unknown', inplace=True)
transfer_history['fee'].fillna('Unknown', inplace=True)
transfer_history['Transfer Fee'].fillna(transfer_history['Transfer Fee'].mean(), inplace=True)

# Convert date and dateUnformatted to datetime
transfer_history['date'] = pd.to_datetime(transfer_history['date'])
# transfer_history['dateUnformatted'] = pd.to_datetime(transfer_history['dateUnformatted'])

In [ ]:
def fill_zero_coordinates(df, tolerance=1e-6):
    # Function to count zeros in the DataFrame
    def count_zeros(df):
        zero_from_lat = (df['from_latitude'].abs() < tolerance).sum()
        zero_from_lon = (df['from_longitude'].abs() < tolerance).sum()
        zero_to_lat = (df['to_latitude'].abs() < tolerance).sum()
        zero_to_lon = (df['to_longitude'].abs() < tolerance).sum()
        return zero_from_lat, zero_from_lon, zero_to_lat, zero_to_lon

    # Count zeros before filling
    before_zeros = count_zeros(df)
    print("Number of zeros before filling:")
    print(f"from_latitude: {before_zeros[0]}, from_longitude: {before_zeros[1]}, to_latitude: {before_zeros[2]}, to_longitude: {before_zeros[3]}")

    # Identify zero coordinates
    zero_from_lat = df['from_latitude'].abs() < tolerance
    zero_from_lon = df['from_longitude'].abs() < tolerance
    zero_to_lat = df['to_latitude'].abs() < tolerance
    zero_to_lon = df['to_longitude'].abs() < tolerance

    # Replace zero coordinates with NaN for easier processing
    df.loc[zero_from_lat, 'from_latitude'] = np.nan
    df.loc[zero_from_lon, 'from_longitude'] = np.nan
    df.loc[zero_to_lat, 'to_latitude'] = np.nan
    df.loc[zero_to_lon, 'to_longitude'] = np.nan

    # Function to get main club name by stripping common feeder team suffixes
    def get_main_club_name(club_name):
        # List of common feeder team suffixes to strip
        common_suffixes = ['U19', 'U17', 'Youth', 'B', 'C', 'II', 'Reserves']
        # Iterate through suffixes and remove them from club_name
        for suffix in common_suffixes:
            club_name = re.sub(rf'\b{suffix}\b', '', club_name, flags=re.IGNORECASE).strip()
        return club_name

    # Apply the function to both 'from' and 'to' club names
    df['main_from_clubName'] = df['from_clubName'].apply(get_main_club_name)
    df['main_to_clubName'] = df['to_clubName'].apply(get_main_club_name)

    # Create a combined DataFrame to fill coordinates
    combined_from = df[['main_from_clubName', 'from_latitude', 'from_longitude']].copy()
    combined_from.rename(columns={'main_from_clubName': 'clubName', 'from_latitude': 'latitude', 'from_longitude': 'longitude'}, inplace=True)

    combined_to = df[['main_to_clubName', 'to_latitude', 'to_longitude']].copy()
    combined_to.rename(columns={'main_to_clubName': 'clubName', 'to_latitude': 'latitude', 'to_longitude': 'longitude'}, inplace=True)

    combined = pd.concat([combined_from, combined_to], ignore_index=True)

    # Fill NaN values with the mean of the respective club's coordinates
    combined['latitude'] = combined.groupby('clubName')['latitude'].transform(lambda x: x.fillna(x.mean()))
    combined['longitude'] = combined.groupby('clubName')['longitude'].transform(lambda x: x.fillna(x.mean()))

    # Fill any remaining NaNs with zero (in case there are clubs with only NaNs)
    combined['latitude'].fillna(0, inplace=True)
    combined['longitude'].fillna(0, inplace=True)

    # Debug prints to check the filled results
    print("Combined DataFrame with filled NaNs:")
    print(combined)

    # Split the combined DataFrame back into from and to parts
    filled_from = combined.iloc[:len(df)]
    filled_to = combined.iloc[len(df):]

    # Merge the filled coordinates back into the original DataFrame
    df['from_latitude'] = filled_from['latitude'].values
    df['from_longitude'] = filled_from['longitude'].values
    df['to_latitude'] = filled_to['latitude'].values
    df['to_longitude'] = filled_to['longitude'].values

    # Count zeros after filling
    after_zeros = count_zeros(df)
    print("Number of zeros after filling:")
    print(f"from_latitude: {after_zeros[0]}, from_longitude: {after_zeros[1]}, to_latitude: {after_zeros[2]}, to_longitude: {after_zeros[3]}")

    return df

In [ ]:
transfer_history_filled = fill_zero_coordinates(transfer_history)

In [ ]:
# Fill missing 'date' values with 'dateUnformatted'
transfer_history_filled['date'] = transfer_history_filled['date'].fillna(transfer_history_filled['dateUnformatted'])

# Fill missing 'Transfer Fee' and 'fee' values with zero
transfer_history_filled['Transfer Fee'] = transfer_history_filled['Transfer Fee'].fillna(0)
transfer_history_filled['fee'] = transfer_history_filled['fee'].fillna(0)

In [ ]:
# Convert datum_mw, highest_date, and last_change to datetime
player_market_value_development['datum_mw'] = pd.to_datetime(player_market_value_development['datum_mw'])
player_market_value_development['highest_date'] = pd.to_datetime(player_market_value_development['highest_date'])
player_market_value_development['last_change'] = pd.to_datetime(player_market_value_development['last_change'])

In [ ]:
# Remove duplicates
player_detail.drop_duplicates(inplace=True)
transfer_history.drop_duplicates(inplace=True)
player_market_value_development.drop_duplicates(inplace=True)

In [ ]:
# Optionally rename columns
player_detail.rename(columns={'Height (m)': 'Height_m'}, inplace=True)
transfer_history.rename(columns={'Transfer Fee': 'Transfer_Fee'}, inplace=True)
player_market_value_development.rename(columns={'Highest market value': 'Highest_market_value'}, inplace=True)

In [ ]:
# player_detail.to_csv('/content/drive/MyDrive/nlp data/Data/player_detail_cleaned.csv')
# transfer_history.to_csv('/content/drive/MyDrive/nlp data/Data/transfer_history_cleaned.csv')
# player_market_value_development.to_csv('/content/drive/MyDrive/nlp data/Data/player_market_value_development_cleaned')

## NER EXTRACTIONS

In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/nlp data/Data/news_feed_all_docs_dropped.csv")

In [ ]:
top_five_content = df['content'].head(5)

# Process each entry with spaCy
for i, content in enumerate(top_five_content):
    doc = nlp(content)
    print(f"Text {i + 1}:")
    print(f"Entities: {[ent.text for ent in doc.ents]}")
    print(f"TOKENS: {[token.text for token in doc]}")
    print(f"POS Tags: {[token.pos_ for token in doc]}")
    print(f"Dependency Labels: {[token.dep_ for token in doc]}")
    print("\n")

In [ ]:
# Process each entry with spaCy and extract names (PERSON) and clubs (ORG)
for i, content in enumerate(top_five_content):
    doc = nlp(content)
    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    clubs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    print(f"Text {i + 1}:")
    print(f"Names: {names}")
    print(f"Clubs: {clubs}")
    print("\n")

In [ ]:
first_content = df['content'].iloc[0]

# Process the first entry with spaCy
doc = nlp(first_content)

# Extract names (PERSON) and clubs (ORG)
names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
clubs = [ent.text for ent in doc.ents if ent.label_ == "ORG"]

# Display the content and extracted entities
print("Content of the first entry:")
print(first_content)
print("\nExtracted names:")
print(names)
print("\nExtracted clubs:")
print(clubs)

In [ ]:
num_chunks = 5
chunk_size = len(df) // num_chunks

# Save each chunk to a separate CSV file
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i < num_chunks - 1 else len(df)

    # Extract the chunk
    chunk = df.iloc[start_idx:end_idx]

    # Save the chunk to a CSV file
    chunk.to_csv(f"/content/drive/MyDrive/nlp data/Data/news_feed_chunk_{i + 1}.csv", index=False)
    print(f"Chunk {i + 1} saved.")

In [ ]:
from tqdm import tqdm

def extract_names(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

def extract_clubs(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == "ORG"]

def extract_fee(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == "MONEY"]

# Initialize tqdm for progress monitoring
tqdm.pandas(desc="Processing content")

# Load the first chunk
chunk_number = 1
chunk = pd.read_csv(f"/content/drive/MyDrive/nlp data/Data/news_feed_chunk_{chunk_number}.csv")

# Apply the functions to the 'content' column with progress monitoring
chunk['names'] = chunk['content'].progress_apply(lambda x: ', '.join(extract_names(x)))
chunk['clubs'] = chunk['content'].progress_apply(lambda x: ', '.join(extract_clubs(x)))
chunk['fee'] = chunk['content'].progress_apply(lambda x: ', '.join(extract_fee(x)))

# Save the processed chunk to a new CSV file
chunk.to_csv(f"/content/drive/MyDrive/nlp data/Data/news_feed_chunk_{chunk_number}_processed.csv", index=False)

# Display the updated DataFrame with new columns
chunk[['content', 'names', 'clubs', 'fee']].head()

## TEXT ANALYSIS FOR NLP

In [ ]:
# miao_feed_all_docs_dropped = pd.read_csv('/content/drive/MyDrive/nlp data/Data/miao_merged_NER_dataset_clean.csv')
# news_feed_all_docs_dropped = pd.read_csv('/content/drive/MyDrive/nlp data/Data/news_merged_NER_dataset_clean.csv')

In [ ]:
# Initialize the inflect engine
p = inflect.engine()

def convert_all_numbers_to_words(text):
    # Pattern to match all numbers
    pattern = re.compile(r'\b\d+\b')

    # Function to replace matched pattern
    def replace(match):
        number = match.group(0)
        return p.number_to_words(number)

    # Replace numbers with words
    return pattern.sub(replace, text)

In [ ]:
# Assuming miao_feed_all_docs_dropped is your DataFrame
miao_feed_all_docs_content = miao_feed_all_docs_dropped['content']

# Define stop words and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

# Tokenize, remove stopwords, punctuation and lemmatize
def process_text(text):
    text = convert_all_numbers_to_words(text)
    tokens = nltk.word_tokenize(text)
    processed_tokens = [
        lemmatizer.lemmatize(token.lower())
        for token in tokens
        if token.lower() not in stop_words and token not in punctuation
        ]
    return processed_tokens

# Measure execution time
start_time = time()

# Apply the process_text function to each entry in the 'content' column
miao_feed_all_docs_dropped['miao_feed_all_docs_processed_tokens'] = miao_feed_all_docs_content.apply(process_text)

# Measure end time and calculate duration
end_time = time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

In [ ]:
miao_feed_all_docs_dropped.head(2)

In [ ]:
#miao_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/nlp data/Data/miao_feed_all_docs_dropped_lemmatize.csv', index=False)

In [ ]:
#Drop keyword column
miao_feed_all_docs_dropped_clean = miao_feed_all_docs_dropped.drop(columns=['content'])

#Rename lemmatized column to token for uniformity
miao_feed_all_docs_dropped_clean.rename(columns={'miao_feed_all_docs_processed_tokens': 'token'}, inplace=True)

In [ ]:
miao_feed_all_docs_dropped_clean.head(2)

In [ ]:
# Measure execution time
start_time = time()

# Assuming miao_feed_all_docs_dropped is your DataFrame
news_feed_all_docs_content = news_feed_all_docs_dropped['content']

# Apply the process_text function to each entry in the 'content' column
news_feed_all_docs_dropped['news_feed_all_docs_processed_tokens'] = news_feed_all_docs_content.apply(process_text)

# Measure end time and calculate duration
end_time = time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

In [ ]:
#news_feed_all_docs_dropped.to_csv('/content/drive/MyDrive/Data/news_feed_all_docs_dropped_lemmatize.csv', index=False)

In [ ]:
#Drop keyword column
news_feed_all_docs_dropped_clean = news_feed_all_docs_dropped.drop(columns=['content'])

#Rename lemmatized column to token for uniformity
news_feed_all_docs_dropped_clean.rename(columns={'news_feed_all_docs_processed_tokens': 'token'}, inplace=True)

In [ ]:
#Combine both news files into one
import pandas as pd

# Concatenate the DataFrames vertically
all_feed_all_docs_clean = pd.concat([news_feed_all_docs_dropped_clean, miao_feed_all_docs_dropped_clean], ignore_index=True)

# Print the concatenated DataFrame
all_feed_all_docs_clean.head()

In [ ]:
all_feed_all_docs_clean.shape

In [ ]:
#all_feed_all_docs_clean.to_csv('/content/drive/MyDrive/nlp data/Data/all_feed_all_docs_clean_token.csv', index=False)

## SENTIMENT ANALYSIS

In [ ]:
# Load the news data
#news_df = pd.read_csv('/content/drive/MyDrive/nlp data/Data/all_feed_all_docs_clean_token.csv')

In [ ]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to get the TextBlob sentiment polarity
def get_textblob_sentiment(text):
    return TextBlob(text).sentiment.polarity


# Function to get the VADER sentiment polarity
def get_vader_sentiment(text):
    return analyzer.polarity_scores(text)['compound']

In [ ]:
# Apply TextBlob sentiment analysis
news_df['textblob_sentiment'] = news_df['token'].apply(get_textblob_sentiment)

In [ ]:
# Apply VADER sentiment analysis
news_df['vader_sentiment'] = news_df['token'].apply(get_vader_sentiment)

# Compare TextBlob and VADER sentiments
print(news_df[['token', 'textblob_sentiment', 'vader_sentiment']].head())

In [ ]:
# Analyze the distribution of sentiments
plt.figure(figsize=(10, 5))
plt.hist(news_df['textblob_sentiment'], bins=20, alpha=0.5, label='TextBlob')
plt.hist(news_df['vader_sentiment'], bins=20, alpha=0.5, label='VADER')
plt.legend(loc='upper left')
plt.title('Sentiment Analysis Distribution')
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#news_df.to_csv('/content/drive/MyDrive/nlp data/Data/all_feed_all_docs_sentiment.csv', index=False)